In [4]:
import pydicom
import numpy as np
import pandas as pd
from skimage import io
from skimage import measure
from skimage.transform import resize
import glob
from tqdm import tqdm
import cv2
import SimpleITK as sitk
from skimage.util import random_noise
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import os
import random
#图片种类
det_class_path = './input/stage_1_detailed_class_info.csv'
#训练集中bbox的位置信息
bbox_path = './input/stage_1_train_labels.csv'
#训练集dcm格式图片
dcm_train_dir = './input/stage_1_train_images/'
#测试集dcm格式图片
dcm_test_dir = './input/stage_1_test_images/'
#测试集png图片
png_test_dir='./input/stage_1_test_png/'
#训练集png图片
png_train_dir='./input/stage_1_train_png/'
#存储npy文件
train_npy='./input/stage_1_train_npy/'
#获取所有dcm图片
dcm_paths =glob.glob('./input/**/*.dcm',recursive=True)
#保存jpg
voc_jpg_dir='./input/VOCdevkit/VOC2007/JPEGImages/'
#保存annotations
voc_anno_dir='./input/VOCdevkit/VOC2007/Annotations/'
#保存txt文件
voc_txt_dir='./input/VOCdevkit/VOC2007/ImageSets/Main'
train_labels =  pd.read_csv(bbox_path)
class_info = pd.read_csv(det_class_path)

In [33]:
#xml文件的开头
head="""<annotation>
	<folder>VOC2007</folder>
	<filename>{0}</filename>
	<source>
		<database>The VOC2007 Database</database>
		<annotation>PASCAL VOC2007</annotation>
		<image>x-ray</image>
		<flickrid>6</flickrid>
	</source>
	<owner>
		<flickrid>rsna</flickrid>
		<name>?</name>
	</owner>
	<size>
		<width>{1}</width>
		<height>{2}</height>
		<depth>{3}</depth>
	</size>
	<segmented>0</segmented>
"""
#xml中bbox的内容
bndbox="""	<object>
		<name>opacity</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>{0}</xmin>
			<ymin>{1}</ymin>
			<xmax>{2}</xmax>
			<ymax>{3}</ymax>
		</bndbox>
	</object>
"""
#xml文件中的结尾
tail="""</annotation>
"""

print(head+bndbox+bndbox+tail)
"""
savename = os.path.join(voc_anno_dir,'test.xml')
f = open(savename, 'w')
ob1 = anno.format('sfda',734,345,3,3,4,5,6)
f.write(ob1)
f.close()
"""

<annotation>
	<folder>VOC2007</folder>
	<filename>{0}</filename>
	<source>
		<database>The VOC2007 Database</database>
		<annotation>PASCAL VOC2007</annotation>
		<image>x-ray</image>
		<flickrid>6</flickrid>
	</source>
	<owner>
		<flickrid>rsna</flickrid>
		<name>?</name>
	</owner>
	<size>
		<width>{1}</width>
		<height>{2}</height>
		<depth>{3}</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>opacity</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>{0}</xmin>
			<ymin>{1}</ymin>
			<xmax>{2}</xmax>
			<ymax>{3}</ymax>
		</bndbox>
	</object>
	<object>
		<name>opacity</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>{0}</xmin>
			<ymin>{1}</ymin>
			<xmax>{2}</xmax>
			<ymax>{3}</ymax>
		</bndbox>
	</object>
</annotation>



"\nsavename = os.path.join(voc_anno_dir,'test.xml')\nf = open(savename, 'w')\nob1 = anno.format('sfda',734,345,3,3,4,5,6)\nf.write(ob1)\nf.close()\n"

In [34]:
#将所有的dcm图像转为png格式，同时存储患有肺炎的图像的numpy数组和bbox的位置信息，以文件的格式保存
for path in tqdm(dcm_paths):
    dcm=sitk.ReadImage(path)
    img_array=sitk.GetArrayFromImage(dcm)[0]
    patientid=path.split('/')[-1][:-4]
    if 'test' in path:
        """
        save_path = os.path.join(voc_jpg_dir,f'{patientid}.jpg')
        file_write_obj = open(os.path.join(voc_txt_dir,'test.txt'), 'a+')
        file_write_obj.writelines(patientid)
        file_write_obj.write('\n')
        file_write_obj.close()
        file_write_obj = open(os.path.join(voc_txt_dir,'opacity_test.txt'), 'a+')
        file_write_obj.writelines(patientid)
        file_write_obj.write('\n')
        file_write_obj.close()
        cv2.imwrite(save_path,img_array)
        """
        
    else:
        labels = train_labels[train_labels['patientId'] == patientid]
        classes = class_info[class_info['patientId'] == patientid]
        #dropna去掉存在空值的记录
        num_label = len(labels.dropna())
        if num_label == 0:
            save_path = os.path.join(voc_jpg_dir,f'{patientid}.jpg')
            """
            cv2.imwrite(save_path,img_array)
            file_write_obj = open(os.path.join(voc_txt_dir,'trainval.txt'), 'a+')
            file_write_obj.writelines(patientid)
            file_write_obj.write('\n')
            file_write_obj.close()
            file_write_obj = open(os.path.join(voc_txt_dir,'opacity_trainval.txt'), 'a+')
            file_write_obj.writelines(patientid+' -1')
            file_write_obj.write('\n')
            file_write_obj.close()
            """
            img_array=cv2.imread(save_path)
            h=head.format(f'{patientid}.jpg',img_array.shape[1],img_array.shape[0],img_array.shape[2])
            anno=h+tail
            savename = os.path.join(voc_anno_dir,f'{patientid}.xml')
            f = open(savename, 'w')
            f.write(anno)
            f.close()
        else:
            save_path = os.path.join(voc_jpg_dir,f'{patientid}.jpg')
            """
            cv2.imwrite(save_path,img_array)
            file_write_obj = open(os.path.join(voc_txt_dir,'trainval.txt'), 'a+')
            file_write_obj.writelines(patientid)
            file_write_obj.write('\n')
            file_write_obj.close()
            file_write_obj = open(os.path.join(voc_txt_dir,'opacity_trainval.txt'), 'a+')
            file_write_obj.writelines(patientid+' 1')
            file_write_obj.write('\n')
            file_write_obj.close()
            """
            img_array=cv2.imread(save_path)
            h=head.format(f'{patientid}.jpg',img_array.shape[1],img_array.shape[0],img_array.shape[2])
            anno=h
            for n in range(num_label):
                label = labels.iloc[n]
                x = int(label['x']) #upper-left
                y = int(label['y']) #upper-left y
                width = int(label['width'])
                height = int(label['height'])
                #bbox coord : (upper-left y,upper-left x,bottom-right y,bottom-right x)
                ymin,xmin,ymax,xmax = [y,x,y+height,x+width]
                b=bndbox.format(xmin,ymin,xmax,ymax)
                anno=anno+b
            anno=anno+tail
            savename = os.path.join(voc_anno_dir,f'{patientid}.xml')
            f = open(savename, 'w')
            f.write(anno)
            f.close()           

  0%|          | 0/26684 [00:00<?, ?it/s]

  0%|          | 10/26684 [00:00<04:44, 93.67it/s]

  0%|          | 19/26684 [00:00<04:52, 91.01it/s]

  0%|          | 26/26684 [00:00<05:30, 80.74it/s]

  0%|          | 33/26684 [00:00<05:42, 77.75it/s]

...

100%|█████████▉| 26669/26684 [12:13<00:00, 36.34it/s]

100%|█████████▉| 26673/26684 [12:13<00:00, 36.34it/s]

100%|█████████▉| 26677/26684 [12:14<00:00, 36.34it/s]

100%|█████████▉| 26681/26684 [12:14<00:00, 36.34it/s]

100%|██████████| 26684/26684 [12:14<00:00, 36.34it/s]

In [47]:
#添加数据增强后的图像信息
npy_path=glob.glob(train_npy+'*.npy')
for path in tqdm(npy_path):
    #if 'rot' not in path and 'shift' not in path:
    #    os.remove(path)
    if 'rot' in path:
        patientid=path.split('/')[-1][:-4]
        #分别获取numpy数组的img和bboxes
        npy = np.load(path).item()
        img = npy['image']
        bboxes = npy['bboxes']
        cv2.imwrite(os.path.join(voc_jpg_dir,f'{patientid}.jpg'),img)
        #添加xml文件
        h=head.format(f'{patientid}.jpg',img_array.shape[1],img_array.shape[0],img_array.shape[2])
        anno=h
        for i in range(len(bboxes)):
            x = int(bboxes[i][0]) #upper-left
            y = int(bboxes[i][1]) #upper-left y
            width = int(bboxes[i][2])
            height = int(bboxes[i][3])
            #bbox coord : (upper-left y,upper-left x,bottom-right y,bottom-right x)
            ymin,xmin,ymax,xmax = [y,x,y+height,x+width]
            b=bndbox.format(xmin,ymin,xmax,ymax)
            anno=anno+b
        anno=anno+tail
        savename = os.path.join(voc_anno_dir,f'{patientid}.xml')
        f = open(savename, 'w')
        f.write(anno)
        f.close()

  0%|          | 0/11318 [00:00<?, ?it/s]

  0%|          | 6/11318 [00:00<03:36, 52.27it/s]

  0%|          | 17/11318 [00:00<02:23, 78.48it/s]

  0%|          | 22/11318 [00:00<02:48, 67.11it/s]

  0%|          | 31/11318 [00:00<02:41, 69.99it/s]

...

100%|█████████▉| 11288/11318 [02:01<00:00, 93.16it/s]

100%|█████████▉| 11297/11318 [02:01<00:00, 93.16it/s]

100%|█████████▉| 11307/11318 [02:01<00:00, 93.16it/s]

100%|█████████▉| 11317/11318 [02:01<00:00, 93.13it/s]

100%|██████████| 11318/11318 [02:01<00:00, 93.14it/s]

In [61]:
#将rot的id信息键入到trainval.txt中
npy_path=glob.glob(train_npy+'*.npy')
for path in tqdm(npy_path):
    #if 'rot' not in path and 'shift' not in path:
    #    os.remove(path)
    if 'rot' in path:
        patientid=path.split('/')[-1][:-4]
        #分别获取numpy数组的img和bboxes
        file_write_obj = open(os.path.join(voc_txt_dir,'trainval.txt'), 'a+')
        file_write_obj.writelines(patientid)
        file_write_obj.write('\n')
        file_write_obj.close()

  0%|          | 0/11318 [00:00<?, ?it/s]

  8%|▊         | 946/11318 [00:00<00:01, 9442.34it/s]

 16%|█▌        | 1772/11318 [00:00<00:01, 8845.75it/s]

 24%|██▍       | 2713/11318 [00:00<00:00, 9033.61it/s]

 31%|███       | 3476/11318 [00:00<00:00, 8679.87it/s]

...

 60%|██████    | 6831/11318 [00:00<00:00, 8343.03it/s]

 70%|███████   | 7929/11318 [00:00<00:00, 8629.29it/s]

 79%|███████▉  | 8977/11318 [00:01<00:00, 8809.45it/s]

 90%|████████▉ | 10162/11318 [00:01<00:00, 9080.90it/s]

100%|██████████| 11318/11318 [00:01<00:00, 9373.81it/s]

In [65]:
#划分val和tain
with open(os.path.join(voc_txt_dir,'trainval.txt'),'r') as r:
    lines=r.readlines()
random.shuffle(lines)
n_valid_samples = 3000
train_lines = lines[n_valid_samples:]
valid_lines = lines[:n_valid_samples]
with open(os.path.join(voc_txt_dir,'train.txt'),'w') as w:
    for l in train_lines:
       w.write(l)
with open(os.path.join(voc_txt_dir,'val.txt'),'w') as w:
    for l in valid_lines:
       w.write(l)

n train samples 29343
n valid samples 3000
31342
31343
1000


In [13]:
import os
with open(os.path.join(voc_txt_dir,'train.txt'),'r') as r:
    lines=r.readlines()
len(lines)
jpg=glob.glob(voc_jpg_dir+'*.jpg')
len(jpg)

16773